In [57]:
'''
1. Linear approximate exon distribution peak VS intron density 
2. Fit gamma to exon length distribution
refined ES parameter parsing
'''
## use ipython parallel programming 
from IPython.parallel import Client
rc = Client()
dview = rc[:]


import pandas as pd; dview.execute("import pandas as pd")
import numpy as np; dview.execute("import numpy as np")


import matplotlib.pylab as plt
import os
%matplotlib inline


# The path where all the GeneMark-ES 4th run models located
modelFilePath = "/home/richard/largeDataSet/ES_Run_modfileAug2015/"
modelFileNames = os.listdir(modelFilePath)

# full data
fulldata = pd.read_csv("../1_DataSmall/fullTableInfoGff3GffRNAESwithDensity20150829.csv")
fulldata.index = fulldata['shortName']

class ESParamParse():
    """Take in 4th run of model file as text and extract info"""
    _DISTR_TYPE= {
        'INITIAL'  :('$INITIAL_DISTR', range(3,10001)),
        'EXON'     :('$EXON_DISTR',range(3,10001)),
        'TERMINAL' : ('$TERMINAL_DISTR', range(3,10001)),
        'SINGLE'   :('$SINGLE_DISTR', range(300,10000)),
        'INTRON'   :('$INTRON_DISTR',range(20,3001)),
        'BP_ACC'   :('$BP_ACC_DISTR',range(2,41)),
        'DON_BP'   :('$DON_BP_DISTR',range(5,3001))
    }
    
    def __init__(self, filename):
        self.f = open(filename)
        
    def maxPosition(self, distrType):
        '''
        return the position with the highest probability
        '''
        # decompose the distrbutionType varibable to distribution type/tag and min_max length range
        TAG, RANGE = self._DISTR_TYPE[distrType]
        # accqure the max length signal
        LENGTH = RANGE[-1]

        # capture length distribution
        lengthData = []
        switch = False
        for line in self.f:
            if switch:
                lengthData.append(line)
            if TAG in line:
                switch = True
            if switch and "{}\t".format(LENGTH) in line:
                switch = False
                break
        lengthData = map(lambda line: line.rstrip().split("\t"), lengthData)
        positionProbabilityPair = map(lambda (a,b): (int(a), float(b)), lengthData)
        df = pd.DataFrame(positionProbabilityPair, columns=['position','probability']) 
        # return position of peak probability
        return df.position[np.argmax(df.probability)]
    
dview.push({'ESParamParse':ESParamParse,'modelFilePath':modelFilePath,'modelFileNames':modelFileNames})

## create data frame that collect all the peak probabilities
shortNames = map(lambda x: x.split(".")[0], modelFileNames)
dfPeakProbs = pd.DataFrame({'shortName': shortNames})

In [56]:
## First of all, find linear relation between exon peak probability and intron density
exonPeakProbMap = dview.map(lambda modelName: ESParamParse(modelFilePath+modelName).maxPosition('EXON'), modelFileNames)
exonPeakProbReduce = exon_map.get()

In [62]:
dfPeakProbs['exonPeakPosition'] = exonPeakProbReduce

In [63]:
dfPeakProbs

,shortName,exonPeakPosition
0,Cersp1,87
1,Ashgo1,1017
2,Myche1,84
3,Sclsc1,96
4,Maggr1,102
5,Basun1,63
6,Terbo2,99
7,Aspsy1,96
8,Stano2,102
9,SerlaS7_9_2,90
